# RatU Day4SD recording info
- No timestamps were deleted after concatenating .dat files from various folders of open-ephys

In [1]:
import numpy as np
import subjects

sess = subjects.nsd.ratUday2[0]
print(sess.recinfo)

filename: /data/Clustering/sessions/RatU/RatUDay2NSD/RatU_Day2NSD_2021-07-24_08-16-38.xml 
# channels: 192
sampling rate: 30000
lfp Srate (downsampled): 1250


In [23]:
sess.neurons.get_neuron_type('pyr').n_neurons

174

# Set probe configuration
- RatU_Day2NSD has two probes: both 128chan-8shanks diagnostic biochips.
- 64 channels in one of the probes (implanted in left hemisphere) had no signal from 4 shanks (probably one of the intan chips was faulty). So only 192 channels were recorded, the channels in .dat file are already order according to depth.


In [ ]:
%matplotlib inline
from neuropy.core import shank, probe, probegroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
badchans = sess.recinfo.skipped_channels

#--- diagnostic-biochip 8 shanks -----------
for i in range(8):
    chans = channel_groups[i]
    shank = shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=15,
        y_shift_per_column=[0,-7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)

probe1 = probe(shanks)

#--- dignostic biochip 4 shanks (4 shanks were bad) ----------- 
shanks = []
for i in range(8,12):

    shank = shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=15,
        y_shift_per_column=[0, -7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)


probe2 = probe(shanks)
probe2.move((probe1.x_max+500,0))

prbgrp = probegroup()
prbgrp.add_probe(probe1)
prbgrp.add_probe(probe2)

prbgrp.filename = sess.fileprefix.with_suffix(".probegroup.npy")
prbgrp.save()
plot_probe(prbgrp)

## Writing to json format for spyking-circus 

In [24]:
from neuropy.utils import probe_util

file = sess.filePrefix.with_suffix('.prb')
probe_util.write_spyking_circus(file,sess.probegroup,shanksCombine=True)


RatU_Day2NSD_2021-07-24_08-16-38.prb file created for Spyking Circus


# Create experimental paradigm
- pre sleep is a little shorter
- animal was lazy on re-maze

In [ ]:
sess.paradigm.to_dataframe()

In [10]:
import pandas as pd
from neuropy.core import Epoch

datetime_data = pd.read_csv(sess.filePrefix.with_suffix('.datetime.csv'))
durations = datetime_data.nFrames/sess.recinfo.dat_sampling_rate
epochs = pd.DataFrame(
    {
        "start": [0,9545,12856,45317],
        "stop": [9544,12855,45316,48496],
        "label": ["pre", "maze", "post",'re-maze'],
    }
)

paradigm = Epoch(epochs=epochs)
paradigm.filename = sess.filePrefix.with_suffix(".paradigm.npy")
paradigm.save()


RatU_Day2NSD_2021-07-24_08-16-38.paradigm.npy saved


In [4]:
durations

0     1313.126400
1     3071.863467
2     5160.814933
3     3311.138133
4    32460.851200
5     3180.740267
Name: nFrames, dtype: float64

In [8]:
1313+3071+5160+3311+32461+3180

48496

# Detect epochs
Here we will various types of epochs which typical for hippocampal recordings.

## Artifacts epochs
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [14]:
%matplotlib widget
from neuropy.analyses import detect_artifact_epochs
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.core import Signal
print(sess)
signal = sess.eegfile.get_signal([23,155])
# filt_trace = signal_process.filter_sig.highpass(signal.traces,cutoff=400)
# filtered_signal = Signal(np.median(filt_trace,axis=0).reshape((1,-1)),signal.sampling_rate)
artifact_epochs = detect_artifact_epochs(signal, thresh=10)
artifact_epochs.filename = sess.filePrefix.with_suffix(".artifact.npy")
artifact_epochs.save()
# plotting.plot_artifact_epochs(artifact_epochs,sess.eegfile.get_signal([175]),downsample_factor=5)


ProcessData(RatU_Day2NSD_2021-07-24_08-16-38.xml)
RatU_Day2NSD_2021-07-24_08-16-38.artifact.npy saved


In [16]:
sess.artifact.to_dataframe()

,start,stop,label,duration
0,1264.6768,1313.1344,,48.4576
1,4277.8808,4384.9976,,107.1168


### write artifact to neuroscope

In [17]:

sess.recinfo.write_epochs(sess.artifact,ext='.art')

### write artifact epochs to spyking circus format

In [19]:
from neuropy.io import SpykingCircusIO

file = sess.filePrefix.with_suffix('.dead')
SpykingCircusIO.write_epochs(file,sess.artifact)

RatU_Day2NSD_2021-07-24_08-16-38.dead created


## Sleep scoring

In [16]:
from neuropy.core import Signal
from neuropy.analyses import brainstates

signal = sess.eegfile.get_signal() 
brainstates = brainstates.detect_brainstates_epochs(signal=signal,probe=sess.probegroup)

starting emg calculation
emg calculation done
channel for sleep detection: 127
spectral properties calculated


In [17]:
brainstates.filename = sess.filePrefix.with_suffix('.brainstates')
brainstates.save()

RatU_Day2NSD_2021-07-24_08-16-38.brainstates saved


## Ripple epochs
To detect ripples one also needs probegroup.

In [ ]:
from neuropy.analyses import oscillations
signal = sess.eegfile.get_signal()
ripple_epochs =oscillations.detect_ripple_epochs(signal, sess.probegroup)
ripple_epochs.filename = sess.filePrefix.with_suffix('.ripple.npy')
ripple_epochs.save()

In [ ]:
signal = sess.eegfile.get_signal(channel_id=[1, 2, 3, 4], t_start=1, t_stop=1.2)
plotting.plot_signal_traces(signal)

# Neurons

## Importing spiketrains from Phy

In [19]:
from neuropy.io import PhyIO
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

cluster_path = Path("/home/bapung/Documents/ClusteringHub/spykcirc/RatU/RatUDay2NSD/RatU_Day2NSD_2021-07-24_08-16-38-1.GUI")
chan_grps = sess.recinfo.channel_groups
phy_data = PhyIO(cluster_path)
spiketrains =phy_data.spiketrains
peak_chans = phy_data.peak_channels
waveforms = phy_data.peak_waveforms
shank_id = sess.probegroup.get_shank_id_for_channels(peak_chans)

neuron_type_id = phy_data.cluster_info.q.values
neuron_type = np.ones(len(neuron_type_id), dtype="U5")
neuron_type[neuron_type_id<4] = 'pyr'
neuron_type[neuron_type_id==6] = 'mua'
neuron_type[neuron_type_id==8] = 'inter'


neurons = Neurons(
    np.array(spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
    waveforms=np.array(waveforms,dtype='object'),
    shank_ids=np.array(shank_id).astype(int),
    neuron_type=neuron_type,
    metadata={'cluster_path':cluster_path}
)

neurons.filename = sess.filePrefix.with_suffix('.neurons')
neurons.save()


RatU_Day2NSD_2021-07-24_08-16-38.neurons saved


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from neuropy.plotting import plot_raster


plt.plot(phy_data.peak_waveforms[0])
plot_raster(neurons,color='jet',add_vert_jitter=True)

## BinnedSpiketrain and Mua objects using Neurons

In [ ]:
mua =sess.neurons.get_mua()
mua.filename = sess.filePrefix.with_suffix(".mua.npy")
mua.save()   


In [ ]:
%matplotlib widget
from neuropy import plotting
smth_mua = sess.mua.get_smoothed(sigma=0.02)
plotting.plot_mua(smth_mua)

In [ ]:
from neuropy.analyses import detect_pbe_epochs

pbe = detect_pbe_epochs(smth_mua)
pbe.filename = sess.filePrefix.with_suffix('.pbe')
pbe.save()


# Position
- concatenated .dat file did not have any deleted timepoints

## Import position from optitrack

In [3]:
from neuropy.io import OptitrackIO
from neuropy.core import Position
from pathlib import Path

opti_folder = sess.filePrefix.parent / 'position'
opti_data = OptitrackIO(dirname=opti_folder)

/data/Clustering/sessions/RatU/RatUDay2NSD/position/Take 2021-07-24 11.11.17 AM.csv
/data/Clustering/sessions/RatU/RatUDay2NSD/position/Take 2021-07-24 09.18.31 PM.csv


## Align position with .dat file

In [8]:
import pandas as pd
from datetime import datetime

#---- startimes of concatenated .dat files
tracking_sRate = opti_data.sampling_rate
rec_datetime = pd.read_csv(sess.filePrefix.with_suffix('.datetime.csv'))
data_time = []
for i, file_time in enumerate(rec_datetime["StartTime"]):
    sync_time = rec_datetime['sync_nframes'][i]/rec_datetime['sync_rate'][i]
    tbegin = datetime.strptime(file_time, "%Y-%m-%d_%H-%M-%S") + pd.Timedelta(sync_time,unit='sec')
    nframes = rec_datetime["nFrames"][i]
    duration = pd.Timedelta(nframes / sess.recinfo.dat_sampling_rate, unit="sec")
    tend = tbegin + duration
    trange = pd.date_range(
        start=tbegin,
        end=tend,
        periods=int(duration.total_seconds() * tracking_sRate),
    )
    data_time.extend(trange)
data_time = pd.to_datetime(data_time)

x,y,z = opti_data.get_position_at_datetimes(data_time)
traces = np.vstack((z,x,y))

position = Position(traces=traces,t_start=0,sampling_rate=opti_data.sampling_rate)

In [9]:
position.filename = sess.filePrefix.with_suffix('.position.npy')
position.save()

RatU_Day2NSD_2021-07-24_08-16-38.position.npy saved


In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

# plt.plot(opti_data.datetime_array,opti_data.z)
plt.plot(sess.position.y,sess.position.x)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
%matplotlib widget
from neuropy.utils import signal_process

signal = sess.eegfile.get_signal([40])

spec = signal_process.SpectrogramBands(signal)


In [16]:
%matplotlib widget

import matplotlib.pyplot as plt

plt.plot(spec.time,spec.theta/500)
plt.plot(sess.position.time,sess.position.y)
plt.plot(sess.position.time[1:],np.diff(sess.position.y)*100)
# plt.xlim([12500,12600])
plt.ylim([-400,400])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-400.0, 400.0)

## Checking position alignment with .dat file
- Comparing theta power, speed and position to check if high theta periods are correlated with the speed of the animal

In [ ]:
%matplotlib widget
from neuropy.utils import signal_process

signal = sess.eegfile.get_signal([158])
spec = signal_process.spectrogramBands(signal.traces[0])

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt

plt.plot(spec.time,spec.theta/1800)
plt.plot(sess.position.time,sess.position.y)
plt.plot(sess.position.time[1:],np.diff(sess.position.x)*100)
# plt.xlim([1500,1600])
plt.ylim([-400,400])